<a href="https://colab.research.google.com/github/edwin-yan/Passion-Fruit-Disease-Detection/blob/andrew/Image_Augmentation_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Augmentation

Using this library to do image augmentation
https://towardsdatascience.com/tensorflow-and-image-augmentation-3610c6c243a2

# SET UP
## Mounting

Note that this assumes that your google drive folder has the same document layout. You must modify if that is not the case.


From your root Google Drive directory create a couple of folders:
* `JHU/DNN/FinalProject/Train_Images`
* `JHU/DNN/FinalProject/Test_Images` 

with Train and test images unzipped in those directories



# **TODO:** GLOBAL VARIABLE set to `True` if using colab and `False` if not!

In [1]:
IS_COLAB = True 

# **TODO:** CHANGE `IMG_DIR` if your google drive directory is different

In [2]:
if IS_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  IMG_PATH = '/content/drive/MyDrive/JHU/DNN/FinalProject'
else:
  IMG_PATH= '.'

Mounted at /content/drive


In [3]:
TRAIN_PATH = f'{IMG_PATH}/Train.csv'
TEST_PATH = f'{IMG_PATH}/Test.csv'

In [4]:
import time

t0 = time.time()

## Imports

In [5]:
import tensorflow as tf

Import tf addons
For some reason colab does not allow minor version e.g. specifying 2.6

In [6]:
!pip -q install -U tensorflow_addons > log 
import tensorflow_addons as tfa

In [7]:
!pip install tf-image
from tf_image.core.bboxes.rotate import random_rotate
from tf_image.core.random import random_function_bboxes
from tf_image.core.bboxes.flip import flip_left_right

In [8]:
# Clone from repo and cd into repo
!git clone https://github.com/edwin-yan/Passion-Fruit-Disease-Detection

# Change directory
%cd Passion-Fruit-Disease-Detection

# Checkout specific branch
!git checkout andrew

Cloning into 'Passion-Fruit-Disease-Detection'...
remote: Enumerating objects: 351, done.
remote: Counting objects: 100% (351/351), done.
remote: Compressing objects: 100% (280/280), done.
remote: Total 351 (delta 146), reused 197 (delta 56), pack-reused 0
Receiving objects: 100% (351/351), 175.31 MiB | 15.44 MiB/s, done.
Resolving deltas: 100% (146/146), done.
/content/Passion-Fruit-Disease-Detection
Branch 'andrew' set up to track remote branch 'andrew' from 'origin'.
Switched to a new branch 'andrew'


In [9]:
!ls

Baseline_Model.ipynb					      Models
Eager_few_shot_od_training_colab_DNN_FINAL.ipynb	      README.md
EDA_colab.ipynb						      Test.csv
EDA.ipynb						      Test_Images
helper.py						      Train_colab.ipynb
Image_Augmentation_colab.ipynb				      Train.csv
inference_colab.ipynb					      Train_Images
Interactive_eager_few_shot_od_training_colab_DNN_FINAL.ipynb  yolo5Input


In [10]:
import pandas as pd
from helper import plot_sample_images, plot_sample_images_multi

In [11]:
import matplotlib.image as img
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
# https://docs.python.org/3/library/uuid.html
import uuid

In [12]:
df_train, df_test = pd.read_csv(TRAIN_PATH), pd.read_csv(TEST_PATH)
print(f"Training Data Shape:  {df_train.shape}\nTest Data Shape:      {df_test.shape}")

Training Data Shape:  (3906, 6)
Test Data Shape:      (931, 1)


In [13]:
df_train.Image_ID.value_counts().head(10)

ID_M34YFAR2    5
ID_AF7K88JS    5
ID_LZI2KVVE    5
ID_OYN556Y0    5
ID_XEMG90OT    5
ID_E71JKA0R    4
ID_NCXRHK4H    4
ID_2OZEHLDV    4
ID_TTWDHCY1    4
ID_I2WILH68    4
Name: Image_ID, dtype: int64

In [14]:
df_train['class'].value_counts()

fruit_healthy      1638
fruit_woodiness    1246
fruit_brownspot    1022
Name: class, dtype: int64

In [15]:
# plot_sample_images_multi(df_train,dir_path=IMG_PATH)

In [16]:
# plot_sample_images_multi(df_test, show_label=False, is_train=False,dir_path=IMG_PATH)

Try Image Augmentation

I think the require format is https://github.com/Ximilar-com/tf-image/blob/master/tf_image/core/bboxes/rotate.py

 [ymin, xmin, ymin, xmax].

## FIRST update the df_train

In [17]:
IMG_HEIGHT = 512
IMG_WIDTH = 512

In [18]:
df_train.head()

,Image_ID,class,xmin,ymin,width,height
0,ID_007FAIEI,fruit_woodiness,87.0,87.5,228.0,311.0
1,ID_00G8K1V3,fruit_brownspot,97.5,17.5,245.0,354.5
2,ID_00WROUT9,fruit_brownspot,156.5,209.5,248.0,302.5
3,ID_00ZJEEK3,fruit_healthy,125.0,193.0,254.5,217.0
4,ID_018UIENR,fruit_brownspot,79.5,232.5,233.5,182.0


Copy for use later

In [19]:
df_original = df_train.copy()

In [20]:
# get xmax
df_train['xmax'] = df_train['xmin'] + df_train['width']
# get ymax
df_train['ymax'] = df_train['ymin'] + df_train['height']

In [21]:
df_train

,Image_ID,class,xmin,ymin,width,height,xmax,ymax
0,ID_007FAIEI,fruit_woodiness,87.0,87.5,228.0,311.0,315.0,398.5
1,ID_00G8K1V3,fruit_brownspot,97.5,17.5,245.0,354.5,342.5,372.0
2,ID_00WROUT9,fruit_brownspot,156.5,209.5,248.0,302.5,404.5,512.0
3,ID_00ZJEEK3,fruit_healthy,125.0,193.0,254.5,217.0,379.5,410.0
4,ID_018UIENR,fruit_brownspot,79.5,232.5,233.5,182.0,313.0,414.5
...,...,...,...,...,...,...,...,...
3901,ID_ZZAB1FH1,fruit_healthy,96.0,175.5,289.5,222.0,385.5,397.5
3902,ID_ZZAB1FH1,fruit_healthy,330.0,241.0,182.0,180.5,512.0,421.5
3903,ID_ZZJZ2CV6,fruit_healthy,358.0,234.0,134.5,107.0,492.5,341.0
3904,ID_ZZJZ2CV6,fruit_healthy,98.5,135.0,275.5,213.5,374.0,348.5


In [22]:
imgID = 'ID_007FAIEI'

In [23]:
def flipLR(bounding_boxes):
  bboxes = bounding_boxes * tf.constant([1, -1, 1, -1], dtype=tf.float32) + tf.stack([0.0, 1.0, 0.0, 1.0])
  bboxes = tf.stack([bboxes[:, 0], bboxes[:, 3], bboxes[:, 2], bboxes[:, 1]], axis=1)
  return bboxes

In [24]:
def rotateImage(imgID,imgClass, xmin_toRot, ymin_toRot,xmax_toRot, ymax_toRot ,df_original):
  train_toRotate_path = f'{IMG_PATH}/Train_Images/{imgID}.jpg'
  image = img.imread(train_toRotate_path)

  xmin_toRot = xmin_toRot
  ymin_toRot = ymin_toRot
  xmax_toRot = xmax_toRot
  ymax_toRot = ymax_toRot

  # 
  # fig, ax = plt.subplots()
  # from matplotlib.patches import Rectangle
  # ax.imshow(image)
  # rect = Rectangle((xmin_toRot,ymin_toRot),xmax_toRot-xmin_toRot,ymax_toRot-ymin_toRot, linewidth=1, edgecolor='r', facecolor='none')

  # ax.add_patch(rect)
  # plt.show()

  image = tf.io.read_file(train_toRotate_path)
  image = tf.image.decode_jpeg(image)
  bounding_boxes = tf.constant([[ymin_toRot,xmin_toRot,  ymax_toRot,xmax_toRot ]],dtype=tf.float32)
  image_rot, bb  = flip_left_right(
      image, bounding_boxes
  )

  bboxes = flipLR(bounding_boxes)
  rotatedBB = (np.array(bboxes))[0]
  rotYmin = rotatedBB[0]
  rotXmin = rotatedBB[1] + 512
  rotYmax = rotatedBB[2]
  rotXmax = rotatedBB[3] + 512
  width = rotXmax - rotXmin
  height = rotYmax - rotYmin

  #
  # fig, ax = plt.subplots()
  # from matplotlib.patches import Rectangle
  # ax.imshow(image_rot)
  # rect = Rectangle((rotXmin,rotYmin),rotXmax-rotXmin,rotYmax-rotYmin, linewidth=1, edgecolor='r', facecolor='none')

  # ax.add_patch(rect)
  # plt.show()

  # Write augmented image to folder: 
  # create uuid based on time
  new_imgID = f'{imgID}_{uuid.uuid1()}'
  tf_image = tf.image.encode_jpeg(image_rot)
  tf.io.write_file(f'{IMG_PATH}/Train_Images_Augmented/{new_imgID}.jpg', tf_image)

  df_original = df_original.append({  'Image_ID':new_imgID,	'class':imgClass,	'xmin':rotXmin,	'ymin':rotYmin,	'width':width,	'height':height},ignore_index=True)

  return df_original, image_rot, rotXmin,  rotYmin, width, height


https://stackoverflow.com/questions/55737105/why-wont-tf-write-file-write-a-file

In [ ]:
for index, row in df_train.iterrows():
    df_original, image_rot,  rotXmin,  rotYmin, width, height  = rotateImage(row['Image_ID'], row['class'], row['xmin'], row['ymin'], row['xmax'],row['ymax'],df_original)
    # fig, ax = plt.subplots()
    # from matplotlib.patches import Rectangle
    # ax.imshow(image_rot)
    # rect = Rectangle((rotXmin,rotYmin),width,height, linewidth=1, edgecolor='r', facecolor='none')

    # ax.add_patch(rect)
    # plt.show()

https://stackoverflow.com/questions/54255431/invalidargumenterror-cannot-compute-matmul-as-input-0zero-based-was-expected

https://stackoverflow.com/questions/37435369/matplotlib-how-to-draw-a-rectangle-on-image

In [ ]:
df_original.to_csv(f'{IMG_PATH}/train_augmented.csv', encoding='utf-8', index=False)